#Грамматика текста как инструмент и инструменты работы с грамматикой

## План

На практической части семинара мы:

- рассмотрим, какие есть библиотеки для работы с грамматикой в тексте
- какие преимущества и недостатки есть у разных вариантов
- какие в целом встречаются проблемы при разметке текста
- посмотрим на конкретных небольших примерах, как строить более сложные запросы с опорой на морфологию и синтаксис

In [2]:
%%capture
!pip install pymorphy2
!pip install Pymystem3
!pip install spacy
!pip install stanza

##Библиотеки

Есть множество библиотек, которые выполняют грамматическую разметку текста. С частью из них вы уже знакомы, потому что они обычно справляются сразу с рядом релевантных для нас задач:

- токенизация
- лемматизация
- определение части речи
- морфологическая разметка
- синтаксическая разметка
- и т. д.

Применение таких библиотек обычно выглядит похоже:

- подгружается анализатор
- создаётся объект, который содержит разметку
- далее с помощью встроенных функций / методов к этой разметке можно обращаться

In [6]:
text = '''В моей комнате, на стене, висит портрет моего приятеля Карла Ивановича Шустерлинга.
Третьего дня, когда я убирал свою комнату, я снял портрет со стены, вытер с него пыль и повесил его обратно.
Потом я отошел, чтобы издали взглянуть, не криво ли он висит. Но когда я взглянул, то у меня похолодели ноги, а волосы встали на голове дыбом.
Вместо Карла Ивановича Шустерлинга на меня глядел со стены страшный, бородатый старик в дурацкой шапочке. Я с криком выскочил из комнаты.'''

In [13]:
#Создаём анализатор
from pymorphy2 import MorphAnalyzer
morph_pymorphy = MorphAnalyzer()

In [14]:
#Анализируем слово
word_pymorphy = morph_pymorphy.parse('комнате')
word_pymorphy

[Parse(word='комнате', tag=OpencorporaTag('NOUN,inan,femn sing,loct'), normal_form='комната', score=0.818181, methods_stack=((DictionaryAnalyzer(), 'комнате', 55, 6),)),
 Parse(word='комнате', tag=OpencorporaTag('NOUN,inan,femn sing,datv'), normal_form='комната', score=0.181818, methods_stack=((DictionaryAnalyzer(), 'комнате', 55, 2),))]

In [18]:
#Выводим информацию о морфологических признаках и лемме
print(word_pymorphy[0].tag)
print(word_pymorphy[0].normal_form)

NOUN,inan,femn sing,loct
комната


In [ ]:
for word in text

In [136]:
#Создаём анализатор
from pymystem3 import Mystem
morph_mystem = Mystem()

In [23]:
#Анализируем текст
text_mystem = morph_mystem.analyze(text)
text_mystem[:5]

[{'analysis': [{'lex': 'в', 'wt': 0.9999917878, 'gr': 'PR='}], 'text': 'В'},
 {'text': ' '},
 {'analysis': [{'lex': 'мой',
    'wt': 1,
    'gr': 'APRO=(пр,ед,жен|дат,ед,жен|род,ед,жен|твор,ед,жен)'}],
  'text': 'моей'},
 {'text': ' '},
 {'analysis': [{'lex': 'комната', 'wt': 1, 'gr': 'S,жен,неод=(пр,ед|дат,ед)'}],
  'text': 'комнате'}]

In [113]:
#Выводим информацию о морфологических признаках и лемме третьего слова
print(text_mystem[4]['analysis'][0]['gr'])
print(text_mystem[4]['analysis'][0]['lex'])

S,жен,неод=(пр,ед|дат,ед)
комната


## pymorphy2 и Pymystem3

###Общая информация

Это библиотеки, разработанные специально для русского языка. pymorphy2 работает непосредственно из Питона. Pymystem3 обращается из Питона к программе [Mystem](https://yandex.ru/dev/mystem/), разработанной Яндексом.

Обе библиотеки опираются на словарь и правила — и выдают для каждого слова те разборы, которые впринципе для них возможны, а также указывают, насколько вероятен каждый из вариантов.

###Что вообще значат эти разборы?

Это сокращённые обозначения морфологических признаков, известных вам из школы: род, число, падеж, время, вид и т. д. Сокращения могут отличаться у разных библиотек: [pymorphy2](https://pymorphy2.readthedocs.io/en/stable/user/grammemes.html#grammeme-docs), [Mystem](https://yandex.ru/dev/mystem/doc/ru/grammemes-values). Работая с конкретной библиотекой, важно сверяться с её разметкой

###Скорость и качество

pymorphy2 работает быстрее, чем Pymystem3 — но качество лучше у последнего.

###Проблема омонимии

Разные разборы для одного и того же слова это проблема. У слова _комнате_ есть два разбора: дательный и предложный падеж. Даже выбирая самый вероятный вариант, мы в части случаев ошибаемся.

Омонимия это в целом большая проблема для анализа текста: разные формы одной леммы (_комнате_) или разные формы разных лемм (_стекло_) могут совпадать. Обычно контекст позволяет однозначно определить, какая форма использована, однако для программ не всегда просто проанализировать этот контекст, чтобы эту форму определить (и снять омонимию).

Перейдём к библиотекам, которые делают синтаксический разбор — а значит смотрят на контекст и могут снимать омонимию.

In [45]:
#Создаём анализатор Spacy, перед этим скачиваем модель
%%capture
!python -m spacy download ru_core_news_sm
import spacy
nlp_spacy = spacy.load("ru_core_news_sm")

In [46]:
#Анализируем текст
text_spacy = nlp_spacy(text)
text

'В моей комнате, на стене, висит портрет моего приятеля Карла Ивановича Шустерлинга.\nТретьего дня, когда я убирал свою комнату, я снял портрет со стены, вытер с него пыль и повесил его обратно.\nПотом я отошел, чтобы издали взглянуть, не криво ли он висит. Но когда я взглянул, то у меня похолодели ноги, а волосы встали на голове дыбом.\nВместо Карла Ивановича Шустерлинга на меня глядел со стены страшный, бородатый старик в дурацкой шапочке. Я с криком выскочил из комнаты.'

In [57]:
#Выводим информацию о морфологических признаках и лемме
print(text_spacy[2].morph)
print(text_spacy[2].lemma_)

Animacy=Inan|Case=Loc|Gender=Fem|Number=Sing
obl


In [71]:
#Выводим информацию о синтаксической функции слова (косвенное дополнение),
#слово, к которому оно относится и его номер в предложении
print(text_spacy[2].dep_)
print(text_spacy[2].head)
print(text_spacy[2].head.i)

obl
висит
7


In [61]:
#Создаём анализатор Stanza
%%capture
import stanza
stanza.download("ru")
nlp_stanza = stanza.Pipeline(lang="ru", processors="tokenize, pos, lemma, depparse")

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: ru (Russian) ...
INFO:stanza:File exists: /root/stanza_resources/ru/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: ru (Russian):
| Processor | Package            |
----------------------------------
| tokenize  | syntagrus          |
| pos       | syntagrus_charlm   |
| lemma     | syntagrus_nocharlm |
| depparse  | syntagrus_charlm   |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Done loading processors!


In [62]:
#Анализируем текст
text_stanza = nlp_stanza(text)
text

'В моей комнате, на стене, висит портрет моего приятеля Карла Ивановича Шустерлинга.\nТретьего дня, когда я убирал свою комнату, я снял портрет со стены, вытер с него пыль и повесил его обратно.\nПотом я отошел, чтобы издали взглянуть, не криво ли он висит. Но когда я взглянул, то у меня похолодели ноги, а волосы встали на голове дыбом.\nВместо Карла Ивановича Шустерлинга на меня глядел со стены страшный, бородатый старик в дурацкой шапочке. Я с криком выскочил из комнаты.'

In [77]:
#Выводим информацию о морфологических признаках и лемме
word_stanza = text_stanza.sentences[0].words[2]
print(word_stanza.feats)
print(word_stanza.lemma)

Animacy=Inan|Case=Loc|Gender=Fem|Number=Sing
комната


In [79]:
#Выводим информацию о синтаксической функции слова (косвенное дополнение),
#слово, к которому оно относится и его номер в предложении
print(word_stanza.deprel)
print(word_stanza.head)
print(text_stanza.sentences[0].words[word_stanza.head - 1].text)

obl
8
висит


##Spacy и Stanza

###Общая информация

Это библиотеки, которые используют для разметки модели машинного обучения. У этого есть ряд достоинств:

- наивысшая точность в настоящий момент;
- снятие неоднозначности с опорой на контекст;
- более качественная работа с редкими / новыми словами, которых нет в словаре.

Но есть и недостатки:

- более долгое время работы;
- влияние обучающей выборки: Spacy обучался на новостных текстах, и лучше всего справляется с ними;
- редко, но могут происходить непредсказуемые ошибки — с которыми словарный метод справился бы без проблем

##Синтаксическая разметка

Сильнее отличается от того синтаксиса, который преподаётся в русской школе. Используется стандарт [Universal Dependencies (UD)](https://universaldependencies.org/ru/index.html).

Синтаксические отношения, которые могут пригодиться:

- nsubj — подлежащее-существительное
- obj — прямое дополнение
- obl — косвенное дополнение, в том числе с предлогом
- amod — прилагательное-определение
- advmod — наречие-обстоятельство
- nmod — зависимое существительное в родительном падеже или с предлогом

In [82]:
#Mystem отработал за 0 секунд

long_text = text * 50
print(len(long_text))
long_text_mystem = morph_mystem.analyze(long_text)

23650


In [83]:
#Spacy отработал за 4 секунды

long_text_spacy = nlp_spacy(long_text)

In [84]:
#Stanza отработала за 52 секунды

long_text_mystem = nlp_stanza(long_text)

##Сравнение библиотек

|  Библиотека |  Метод | Качество  |  Скорость | Требовательность к ресурсам  |
|:---|:---:|:---:|:---:|:---:|
| pymorphy2  |  правила | среднее  | высокая  | очень низкая |
|  Pymystem3 | правила  | высокое  | высокая  | низкая  |
|  Spacy |  машинное обучение |  высокое | средняя  | средняя  |
|  Stanza |  машинное обучение |  очень высокое | низкая  | высокая  |

## Обработка текстов большего размера

Разметим набор текстов большего объёма. Возьмём в качестве примера несколько сотен новостей с сайта Панорама.

In [91]:
#Работа с таблицами — понадобится, чтобы хранить наш корпус
#Больше подробностей об этом инструменте на следующем семинаре
import pandas as pd

#Выполнение быстрых математических вычислений — пригодится, потому что иначе
#корпус с большим количеством текстов может долго обрабатываться
import numpy as np

In [92]:
#Считываем из файла корпус новостей, который лежит по ссылке

corpus_link = 'https://raw.githubusercontent.com/knapweedss/TextMining_HSE/main/sem3/panorama_corpus.tsv'

panorama_corpus = pd.read_csv(corpus_link, sep='\t')

In [93]:
panorama_corpus.head()

,index,date,sphere,title,text,link
0,0,4-3-2023,Общество,Гражданам разрешили добывать нефть и газ на св...,Государственная дума приняла в третьем чтении ...,https://panorama.pub/news/grazdanam-razresili-...
1,1,4-3-2023,Политика,Китай подал заявку на вступление в Союзное гос...,Заведующий Канцелярией Комиссии ЦК КПК по инос...,https://panorama.pub/news/kitaj-podal-zaavku-n...
2,2,4-3-2023,Политика,Деколонизаторки из Франции потребовали вернуть...,Активисты движения «Смерть колониализму» из Па...,https://panorama.pub/news/dekolonizatorki-iz-f...
3,3,4-3-2023,Политика,Победа России: Госдума пересмотрела результаты...,Государственная дума приняла постановление «Об...,https://panorama.pub/news/gosduma-peresmotrela...
4,4,3-3-2023,Общество,Бездетных россиян будут ежегодно штрафовать,Государственная дума приняла в первом чтении з...,https://panorama.pub/news/bezdetnyh-rossian-pr...


In [98]:
#Для начала поработаем с одной из новостей. Будем использовать Spacy
#По размеченному тексту можно итерировать: идти слово за словом и получать необходимую разметку

news_nlp = nlp_spacy(panorama_corpus['text'][0])

#Идём в цикле по одному слову
#Функция enumerate() считает слова изаписывает номер в переменную word_index
for word_index, word in enumerate(news_nlp):
    print([word, word.lemma_, word.pos_, str(word.morph), word.dep_, word.head])
    if word_index >= 10:
      break

[Государственная, 'государственный', 'ADJ', 'Case=Nom|Degree=Pos|Gender=Fem|Number=Sing', 'amod', дума]
[дума, 'дума', 'NOUN', 'Animacy=Inan|Case=Nom|Gender=Fem|Number=Sing', 'nsubj', приняла]
[приняла, 'принять', 'VERB', 'Aspect=Perf|Gender=Fem|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act', 'ROOT', приняла]
[в, 'в', 'ADP', '', 'case', чтении]
[третьем, 'третий', 'ADJ', 'Case=Loc|Degree=Pos|Gender=Neut|Number=Sing', 'amod', чтении]
[чтении, 'чтение', 'NOUN', 'Animacy=Inan|Case=Loc|Gender=Neut|Number=Sing', 'obl', приняла]
[законопроект«О, 'законопроект«о', 'NOUN', 'Animacy=Inan|Case=Acc|Gender=Masc|Number=Sing', 'nmod', чтении]
[внесении, 'внесение', 'NOUN', 'Animacy=Inan|Case=Loc|Gender=Neut|Number=Sing', 'obj', приняла]
[изменении, 'изменение', 'NOUN', 'Animacy=Inan|Case=Loc|Gender=Neut|Number=Sing', 'nmod', внесении]
[в, 'в', 'ADP', '', 'case', порядок]
[порядок, 'порядок', 'NOUN', 'Animacy=Inan|Case=Acc|Gender=Masc|Number=Sing', 'nmod', изменении]


In [99]:
#Делаем то же самое — но для каждой из новостей, и записываем всю разметку в список
#Также записываем номер каждого слова и новости — это пригодится нам потом

words = []

for text_index, row in enumerate(panorama_corpus['text']):
  news_nlp = nlp_spacy(row)
  for word_index, word in enumerate(news_nlp):
    words.append([word_index, text_index, word, word.lemma_, word.pos_,
                  str(word.morph), word.dep_, word.head])

In [107]:
words[:2]

[[0,
  0,
  Государственная,
  'государственный',
  'ADJ',
  'Case=Nom|Degree=Pos|Gender=Fem|Number=Sing',
  'amod',
  дума],
 [1,
  0,
  дума,
  'дума',
  'NOUN',
  'Animacy=Inan|Case=Nom|Gender=Fem|Number=Sing',
  'nsubj',
  приняла]]

In [150]:
#Добавляем названия столбцов будущей таблицы

cols = ['index', 'text_index', 'word', 'lemma', 'pos', 'morph',
        'synt_relation', 'head']

#Превращаем в таблицу полученный текст

panorama_by_word = pd.DataFrame(words, columns = cols)

In [151]:
#Теперь у нас есть таблица, где каждое слово размечено!
#Уже можно представить, как с ней работать, но конкретные инструменты обсудим на следующей паре

panorama_by_word.head()

,index,text_index,word,lemma,pos,morph,synt_relation,head
0,0,0,Государственная,государственный,ADJ,Case=Nom|Degree=Pos|Gender=Fem|Number=Sing,amod,2
1,1,0,дума,дума,NOUN,Animacy=Inan|Case=Nom|Gender=Fem|Number=Sing,nsubj,3
2,2,0,приняла,принять,VERB,Aspect=Perf|Gender=Fem|Mood=Ind|Number=Sing|Te...,root,0
3,3,0,в,в,ADP,None,case,6
4,4,0,третьем,третий,ADJ,Case=Loc|Degree=Pos|Gender=Neut|Number=Sing,amod,6


Далее аналогичная разметка с помощью других анализаторов — на случай, если при вашей работе покажется полезным использовать какой-то другой

In [111]:
#Аналогичная разметка с помощью pymorphy (берём самый вероятный вариант разметки;
#без синтаксиса; удаляем пунктуацию)

import re

cols = ['index', 'text_index', 'word', 'lemma', 'morph']
words = []

for text_index, row in enumerate(panorama_corpus['text']):

  row = re.sub('[^a-zа-я]+', ' ', text.lower())

  for word_index, word in enumerate(row.split()):
    word_parsed = morph_pymorphy.parse(word)[0]
    words.append([word_index, text_index, word, word_parsed.normal_form, word_parsed.tag])

panorama_by_word = pd.DataFrame(words, columns = cols)

In [142]:
#Аналогичная разметка с помощью Mystem (берём самый вероятный вариант разметки;
#без синтаксиса; удаляем пунктуацию)

cols = ['index', 'text_index', 'word', 'lemma', 'morph']
words = []

for text_index, row in enumerate(panorama_corpus['text']):
  news_nlp = morph_mystem.analyze(row)
  for word_index, word in enumerate(news_nlp):
    try:
      words.append([word_index, text_index, word['text'], word['analysis'][0]['lex'],
                  word['analysis'][0]['gr']])
    except KeyError:
      if word['text'] == ' ':
        continue
      else:
        words.append([word_index, text_index, word['text'], '', ''])
    except IndexError:
      words.append([word_index, text_index, word['text'], 'NA', 'NA'])

panorama_by_word = pd.DataFrame(words, columns = cols)

In [ ]:
#Аналогичная разметка с помощью Stanza (работает долго)

cols = ['index', 'text_index', 'word', 'lemma', 'pos', 'morph',
        'synt_relation', 'head']
words = []

for text_index, row in enumerate(panorama_corpus['text']):
  news_nlp = nlp_stanza(row)
  for word_index, word in enumerate(news_nlp.iter_words()):
    words.append([word_index, text_index, word.text, word.lemma, word.upos, word.feats,
                  word.deprel, word.head])

panorama_by_word = pd.DataFrame(words, columns = cols)